In [1]:
#%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import marshaltools
from ast import literal_eval
import logging
from astropy.time import Time
from ipywidgets import interactive
import ipywidgets as widgets
import re, os
import requests
import sncosmo
import json

from utils import get_config


/home/jnordin/local/miniconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jnordin/local/miniconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [53]:
# If you are running this notebook for the first time, you will be prompted to enter your username and
# the path to your downloads from slack. If you entered something incorrectly, you can correct this in .config
config = get_config()

username = config['username']
date = '2019-01-03'
maxz = 0.1
minpeakmag = 19.5   # A candidate need to have at least one detection brigther than this
mindet = 4          # A candidate need to have at least this many detections
maxage = 30         # If a detection has an age older than this, skip (stars,age). 
                    # Q: How can histories be older than 30days?
minfilters = 2      # Reported detections in at least this many filters
minrefframes = 0    # Cut away alert information where this is not positive (how can it not be, but ok...)

marshal_saveprogs = 'AMPEL'   # Candidates already in this program will not be scanned. Reg exp string, eg 'AMPEL|RCF'
marshal_savid = 42
logpath = 'logs/'
slackfilepath = config['slackfilepath']

# Get the other set of marshal source ids. Lets not talk about, took too much of my life alrady
with open('sncandid_soup.json', 'r') as fp:
    soupids = json.load(fp)

# log. This should be saved and pushed to github. If everyone does this we should have the full scanning
# history for this run
logger = logging.getLogger(username)
handler = logging.FileHandler(os.path.join(logpath, 'ztfcosmo_%s_%s.log'%(date,username)))
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)



In [54]:
df_sum = pd.read_csv(os.path.join(slackfilepath, "Summary_%s.csv"%(date)))
df_phot = pd.read_csv(os.path.join(slackfilepath, "Photometry_%s.csv"%(date)))
# New version has started using MISSING instead of something else...
df_sum.replace(to_replace="MISSING",value=np.NaN, inplace=True)
df_phot.replace(to_replace="MISSING",value=np.NaN, inplace=True)

if minrefframes>0:
    df_phot = df_phot[df_phot["nframesref"]>minrefframes]
    logger.info("Cutting down alert info to subset with nrefframes keyword")

/home/jnordin/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (108,109,110,115,116,117,119,122,123,124,125,126,127,128,129,130,131,132,133,134,153) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [55]:
logger.info("There were %s candidates reported %s"%(df_sum.shape[0],date))

INFO:jnordin:There were 1122 candidates reported 2019-01-03


In [56]:
# Find already classified SNe
i_untyped = (df_sum["T2-classification"]== "None") | (df_sum["T2-classification"].isnull())
df_classified = df_sum[ ~i_untyped]
df_tmp = df_sum[ i_untyped]
logger.info("Out of these %s already have Marshal type and will be skipped."%len(df_classified))


INFO:jnordin:Out of these 207 already have Marshal type and will be skipped.


In [57]:
# Are there any remaining candidates with spectroscopic redshifts above limit
i_highz = ( (df_tmp['T2-NEDz_z'].astype(float)>maxz) | (df_tmp['T2-milliquas_redshift'].astype(float)>maxz)  | (df_tmp['T2-SDSS_spec_z'].astype(float)>maxz) ) 
df_classified = df_tmp[ i_highz]
df_tmp2 = df_tmp[~i_highz]
logger.info("%s have a catalog z larger than %s  and will be skipped."%(np.sum(i_highz),maxz))


INFO:jnordin:286 have a catalog z larger than 0.1  and will be skipped.


In [58]:
# Which are already saved in the marshal?
i_saved = df_tmp2["T2-programs"].str.contains(marshal_saveprogs, regex=True, na=False, flags=re.IGNORECASE)
df_saved = df_tmp2[i_saved]
df_candidates = df_tmp2[~i_saved]
logger.info("Detected %s already saved candidates and %s new"%(len(df_saved),len(df_candidates)))

INFO:jnordin:Detected 152 already saved candidates and 477 new


In [59]:
# In case we actually wish to scan all (marshal is down)
df_candidates = df_tmp2

In [60]:
# Examine new candidates, build a list of the onces fulfilling basic criteria
sncandidates = []
for sne in df_candidates.iterrows():
    print( sne[1]["ztf_name"] )
    phot = df_phot[ df_phot["ztf_name"]==sne[1]["ztf_name"] ]
    filters = phot["fid"]
    jd = phot["jd"]
    magpsf = phot['magpsf']
    if np.min(magpsf)> minpeakmag:
        msg = "%s: Skipping %s, never reached limit %s"%(date,sne[1]["ztf_name"],minpeakmag)
        logger.info(msg)
        print(msg)
        continue
    if len(magpsf)< mindet:
        msg = "%s: Skipping %s, less than %s detections"%(date,sne[1]["ztf_name"],mindet)
        logger.info(msg)
        print(msg)
        continue

    age = np.max(jd)-np.min(jd)
    if age>maxage:
        msg = "%s: Skipping %s, age %s longer than limit %s"%(date,sne[1]["ztf_name"],age,maxage)
        logger.info(msg)
        print(msg)
        continue
        
    phot = df_phot[ df_phot["ztf_name"]==sne[1]["ztf_name"] ]
    if len(np.unique(phot["fid"]))<minfilters:
        msg = "%s: Skipping %s, only %s filters"%(date,sne[1]["ztf_name"],len(np.unique(phot["fid"])))
        logger.info(msg)
        print(msg)
        continue
        
    sncandidates.append(sne[1]["ztf_name"])
    continue
        
    

print("Left with %s candidates to scan"%(len(sncandidates)))

INFO:jnordin:2019-01-03: Skipping ZTF17aaaakga, age 62.73217590013519 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF17aaaazsd, age 63.79555549984798 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF17aaabsxa, age 58.73417819989845 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF17aaaewpx, age 30.845590200275183 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF17aaagtfh, age 63.98533560009673 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF17aaagtqc, age 63.98019680008292 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF17aaajazz, age 30.894351799972355 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF17aaajssx, age 58.8605554997921 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF17aaajxnq, age 61.860694400500506 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF17aaakpvi, less than 4 detections
INFO:jnordin:2019-01-03: Skipping ZTF17aaanqxz, age 64.0482755000703 longer than limit 30
INFO:jnordin:2019-

ZTF17aaaakga
2019-01-03: Skipping ZTF17aaaakga, age 62.73217590013519 longer than limit 30
ZTF17aaaazsd
2019-01-03: Skipping ZTF17aaaazsd, age 63.79555549984798 longer than limit 30
ZTF17aaabsxa
2019-01-03: Skipping ZTF17aaabsxa, age 58.73417819989845 longer than limit 30
ZTF17aaaewpx
2019-01-03: Skipping ZTF17aaaewpx, age 30.845590200275183 longer than limit 30
ZTF17aaagtfh
2019-01-03: Skipping ZTF17aaagtfh, age 63.98533560009673 longer than limit 30
ZTF17aaagtqc
2019-01-03: Skipping ZTF17aaagtqc, age 63.98019680008292 longer than limit 30
ZTF17aaajazz
2019-01-03: Skipping ZTF17aaajazz, age 30.894351799972355 longer than limit 30
ZTF17aaajssx
2019-01-03: Skipping ZTF17aaajssx, age 58.8605554997921 longer than limit 30
ZTF17aaajxnq
2019-01-03: Skipping ZTF17aaajxnq, age 61.860694400500506 longer than limit 30
ZTF17aaakpvi
2019-01-03: Skipping ZTF17aaakpvi, less than 4 detections
ZTF17aaanqxz
2019-01-03: Skipping ZTF17aaanqxz, age 64.0482755000703 longer than limit 30
ZTF17aaaomuj
ZTF17

INFO:jnordin:2019-01-03: Skipping ZTF18aaakpmq, age 51.86079860012978 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF18aaapjfq, age 47.12076390022412 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF18aaapnul, age 51.862395900301635 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF18aaaqyxu, age 30.94453710038215 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF18aaarxlx, age 30.959525499958545 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF18aaasazo, age 36.01097229961306 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF18aabfjxf, age 60.805775499902666 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF18aabfznw, age 63.76988429995254 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF18aabgmjt, age 59.998205999843776 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF18aabgnfo, age 64.05055550020188 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF18aabgsjq, age 36.05872690072283 longer than limi

ZTF18aaakpmq
2019-01-03: Skipping ZTF18aaakpmq, age 51.86079860012978 longer than limit 30
ZTF18aaapjfq
2019-01-03: Skipping ZTF18aaapjfq, age 47.12076390022412 longer than limit 30
ZTF18aaapnul
2019-01-03: Skipping ZTF18aaapnul, age 51.862395900301635 longer than limit 30
ZTF18aaaqyxu
2019-01-03: Skipping ZTF18aaaqyxu, age 30.94453710038215 longer than limit 30
ZTF18aaarxlx
2019-01-03: Skipping ZTF18aaarxlx, age 30.959525499958545 longer than limit 30
ZTF18aaasazo
2019-01-03: Skipping ZTF18aaasazo, age 36.01097229961306 longer than limit 30
ZTF18aaazsiz
ZTF18aabfjxf
2019-01-03: Skipping ZTF18aabfjxf, age 60.805775499902666 longer than limit 30
ZTF18aabfxpw
ZTF18aabfznw
2019-01-03: Skipping ZTF18aabfznw, age 63.76988429995254 longer than limit 30
ZTF18aabgmjt
2019-01-03: Skipping ZTF18aabgmjt, age 59.998205999843776 longer than limit 30
ZTF18aabgnfo
2019-01-03: Skipping ZTF18aabgnfo, age 64.05055550020188 longer than limit 30
ZTF18aabgsjq
2019-01-03: Skipping ZTF18aabgsjq, age 36.05872

INFO:jnordin:2019-01-03: Skipping ZTF18aajofqx, age 46.0817824001424 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF18aajvwbe, age 59.012916599866 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF18aajytjt, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF18aakcfak, age 58.943240699823946 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF18aakfabl, age 62.95478010037914 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF18aakoejd, less than 4 detections
INFO:jnordin:2019-01-03: Skipping ZTF18aamrkmd, age 48.99918989976868 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF18aarcatu, age 55.97722219955176 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF18aarltmo, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF18aasszwr, age 63.04187500011176 longer than limit 30
INFO:jnordin:2019-01-03: Skipping ZTF18aasxdqi, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF18aaujlmk, age 63.04187500011176 longe

...

ZTF18adbzmfe
2019-01-03: Skipping ZTF18adbzmfe, never reached limit 19.5
ZTF18adbzzco
2019-01-03: Skipping ZTF18adbzzco, never reached limit 19.5
ZTF18adcagrn
2019-01-03: Skipping ZTF18adcagrn, less than 4 detections
ZTF18adcakbo
2019-01-03: Skipping ZTF18adcakbo, less than 4 detections
ZTF18adcapvj
2019-01-03: Skipping ZTF18adcapvj, never reached limit 19.5
ZTF18adcarcc
ZTF18adcarey
2019-01-03: Skipping ZTF18adcarey, never reached limit 19.5
ZTF18adcbaxc
ZTF18adcbkiq
2019-01-03: Skipping ZTF18adcbkiq, never reached limit 19.5
ZTF18adcbnsb
ZTF18adcbwln
2019-01-03: Skipping ZTF18adcbwln, never reached limit 19.5
ZTF18adcbyna
2019-01-03: Skipping ZTF18adcbyna, less than 4 detections
ZTF18adcbytv
2019-01-03: Skipping ZTF18adcbytv, never reached limit 19.5
ZTF18adccdba
2019-01-03: Skipping ZTF18adccdba, never reached limit 19.5
ZTF18adccgir
2019-01-03: Skipping ZTF18adccgir, less than 4 detections
ZTF18adccxna
2019-01-03: Skipping ZTF18adccxna, never reached limit 19.5
ZTF18adcdfga
2019-01

INFO:jnordin:2019-01-03: Skipping ZTF19aaactno, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF19aaacuzl, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF19aaacvnr, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF19aaacvrx, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF19aaacvsc, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF19aaadfnn, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF19aaadfno, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF19aaadgjf, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF19aaadxqs, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF19aaadxrf, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF19aaaeacg, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF19aaaeafd, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF19aaaeetf, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF19aaaefkt, nev

2019-01-03: Skipping ZTF19aaactcw, less than 4 detections
ZTF19aaactno
2019-01-03: Skipping ZTF19aaactno, never reached limit 19.5
ZTF19aaacuzl
2019-01-03: Skipping ZTF19aaacuzl, never reached limit 19.5
ZTF19aaacvnr
2019-01-03: Skipping ZTF19aaacvnr, never reached limit 19.5
ZTF19aaacvrx
2019-01-03: Skipping ZTF19aaacvrx, never reached limit 19.5
ZTF19aaacvsc
2019-01-03: Skipping ZTF19aaacvsc, never reached limit 19.5
ZTF19aaadfnn
2019-01-03: Skipping ZTF19aaadfnn, never reached limit 19.5
ZTF19aaadfno
2019-01-03: Skipping ZTF19aaadfno, never reached limit 19.5
ZTF19aaadgjf
2019-01-03: Skipping ZTF19aaadgjf, never reached limit 19.5
ZTF19aaadxmz
ZTF19aaadxqs
2019-01-03: Skipping ZTF19aaadxqs, never reached limit 19.5
ZTF19aaadxrf
2019-01-03: Skipping ZTF19aaadxrf, never reached limit 19.5
ZTF19aaaeacg
2019-01-03: Skipping ZTF19aaaeacg, never reached limit 19.5
ZTF19aaaeafd
2019-01-03: Skipping ZTF19aaaeafd, never reached limit 19.5
ZTF19aaaeetf
2019-01-03: Skipping ZTF19aaaeetf, never

INFO:jnordin:2019-01-03: Skipping ZTF19aaaffzf, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF19aaafgsx, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF19aaafgwv, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF19aaafhbh, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF19aaafhft, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF19aaafkwm, never reached limit 19.5
INFO:jnordin:2019-01-03: Skipping ZTF19aaafkxz, never reached limit 19.5


2019-01-03: Skipping ZTF19aaaffln, never reached limit 19.5
ZTF19aaaffzf
2019-01-03: Skipping ZTF19aaaffzf, never reached limit 19.5
ZTF19aaafgsx
2019-01-03: Skipping ZTF19aaafgsx, never reached limit 19.5
ZTF19aaafgwv
2019-01-03: Skipping ZTF19aaafgwv, never reached limit 19.5
ZTF19aaafhbh
2019-01-03: Skipping ZTF19aaafhbh, never reached limit 19.5
ZTF19aaafhft
2019-01-03: Skipping ZTF19aaafhft, never reached limit 19.5
ZTF19aaafkwm
2019-01-03: Skipping ZTF19aaafkwm, never reached limit 19.5
ZTF19aaafkxz
2019-01-03: Skipping ZTF19aaafkxz, never reached limit 19.5
Left with 150 candidates to scan


In [61]:
# Function to make stupid plot of a SN lightcurve
def plotty(i):
    """
    Quick plot of SN nbr k
    i : {0:undecided, 1:submit, 2:wait 3:bogus}
    """
    
    global snnbr
    
    
    # Evaluate reply
    snname = sncandidates[snnbr]
    if i == 'Wait':
        logger.info("Waiting for further obs for SN %s %s"%(snnbr,snname))
        decisions[snnbr] = i
        snnbr += 1
    elif i == 'Submit':
        logger.info("Sending SN %s %s for possible follow-up"%(snnbr,snname))
        decisions[snnbr] = i
        snnbr += 1

    elif i == 'Garbage':
        logger.info("I never want to see SN %s %s again!"%(snnbr,snname))
        decisions[snnbr] = i
        snnbr += 1

    elif i == 'GoBack':
        logger.info("I want to scan some more, go back!")
        snnbr -= 1
    elif i=='Nothing':
        # Lets do nothing
        pass
    
    # Are we done?
    if snnbr==len(sncandidates):
        print( "Seems like we are all done. How do we exit?")
        return False
    
    
    # Reset    
    snname = sncandidates[snnbr]
    

    phot = df_phot[ df_phot["ztf_name"]==snname ]
    filters = phot["fid"]
    jd = phot["jd"]
    magpsf = phot['magpsf']


    plt.title(snname + ' ' + str(snnbr+1) + ' out of '+str(len(sncandidates)))
    plt.plot(jd[filters==1]-Time.now().jd,magpsf[filters==1],'go')
    plt.plot(jd[filters==2]-Time.now().jd,magpsf[filters==2],'ro')
    plt.plot(jd[filters==3]-Time.now().jd,magpsf[filters==3],'ko')
    plt.gca().invert_yaxis()
    plt.show()
    wiggy.value = 'Nothing'
    
    return (i)
wiggy = widgets.RadioButtons(
    options=['Nothing','Wait', 'Submit', 'Garbage','GoBack'],
    value='Nothing',
    description='Action:',
    disabled=False
)
y = interactive(plotty,i = wiggy)

In [62]:
# Presumably you want to scan from the first SN, but you can change this and run display again to revisit something
# You cant jump ahead in the list though
snnbr = 0


In [63]:
# This is the all important list of decisions you have made
decisions = {}

In [64]:
# This is the scanning box! 
# You have three choices (Nothing is not a choice and GoBack steps lets you go back in order)
# - Wait : Select this if the transient is rizing and might get into RCF range (~<18.7)
# - Submit : Should probably get a spectrum (you do not need to worry about where just yet)
# - Garbage : Variable star or clearly non SNIa. Evanetually these will be rejected and never more show up
display(y)

interactive(children=(RadioButtons(description='Action:', options=('Nothing', 'Wait', 'Submit', 'Garbage', 'Go…

In [65]:
# Save the full decisions to the log
logger.info("Full decision set: %s"%(decisions))
for dec_number, dec_type in decisions.items():
    if dec_type=='Submit':
        print((sncandidates[dec_number]))
        print("https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=%s&channel=HU_GP_CLEAN"%(sncandidates[dec_number]))

INFO:jnordin:Full decision set: {0: 'Wait', 1: 'Wait', 2: 'Wait', 3: 'Wait', 4: 'Wait', 5: 'Wait', 6: 'Wait', 7: 'Wait', 8: 'Wait', 9: 'Wait', 10: 'Wait', 11: 'Wait', 12: 'Wait', 13: 'Wait', 14: 'Wait', 15: 'Wait', 16: 'Wait', 17: 'Wait', 18: 'Wait', 19: 'Wait', 20: 'Wait', 21: 'Wait', 22: 'Wait', 23: 'Wait', 24: 'Wait', 25: 'Wait', 26: 'Wait', 27: 'Wait', 28: 'Wait', 29: 'Wait', 30: 'Wait', 31: 'Wait', 32: 'Wait', 33: 'Wait', 34: 'Wait', 35: 'Wait', 36: 'Wait', 37: 'Wait', 38: 'Wait', 39: 'Wait', 40: 'Wait', 41: 'Wait', 42: 'Wait', 43: 'Wait', 44: 'Wait', 45: 'Wait', 46: 'Wait', 47: 'Wait', 48: 'Wait', 49: 'Wait', 50: 'Wait', 51: 'Wait', 52: 'Submit', 53: 'Wait', 54: 'Wait', 55: 'Wait', 56: 'Wait', 57: 'Wait', 58: 'Wait', 59: 'Wait', 60: 'Wait', 61: 'Submit', 62: 'Wait', 63: 'Wait', 64: 'Submit', 65: 'Submit', 66: 'Wait', 67: 'Wait', 68: 'Wait', 69: 'Wait', 70: 'Wait', 71: 'Wait', 72: 'Wait', 73: 'Submit', 74: 'Submit', 75: 'Wait', 76: 'Wait', 77: 'Wait', 78: 'Submit', 79: 'Wait', 80:

ZTF18acvgjqv
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18acvgjqv&channel=HU_GP_CLEAN
ZTF18acvwcky
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18acvwcky&channel=HU_GP_CLEAN
ZTF18acvwdkk
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18acvwdkk&channel=HU_GP_CLEAN
ZTF18acvwmpw
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18acvwmpw&channel=HU_GP_CLEAN
ZTF18acwwods
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18acwwods&channel=HU_GP_CLEAN
ZTF18acwyaws
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18acwyaws&channel=HU_GP_CLEAN
ZTF18acxgqxq
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18acxgqxq&channel=HU_GP_CLEAN
ZTF18acxhqur
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18acxhqur&channel=HU_GP_CLEAN
ZTF18acywbgz
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18ac

In [34]:
# Now fun starts for real. Lets download all the candidates and check whether we find all the candidate ids there
# (cause are the only that are useful for saving). This will take a while.
pl = marshaltools.ProgramList('AMPEL Test',load_candidates=True)
cpl = marshaltools.ProgramList('Cosmology',load_candidates=False)


ERROR:marshaltools.ProgramList:Error 400: Bad Request: The request was invalid. An accompanying error message will explain why.
ERROR:marshaltools.ProgramList:No json returned: status 400


TypeError: 'int' object is not iterable

In [15]:
ingestoldcand_start = Time.now()
toingest = []
for snnbr, choice in decisions.items():
    if not choice=='Submit' : continue
        
    if sncandidates[snnbr] in cpl.sources:
        print("%s already saved in cosmology!"%(sncandidates[snnbr]))

    avroid =  np.max( df_phot['_id'][ df_phot["ztf_name"]==sncandidates[snnbr] ]  )
    logger.info("Trying to ingest %s through avroid %s"%(sncandidates[snnbr],avroid))
    canddate = pl.find_source(sncandidates[snnbr],include_candidates=True)
    if not canddate is None:
        logger.info('... already a candidate with id %s'%(canddate['candid']))
        continue
    toingest.append(avroid)

    
print("Starting ingesting at %s (useful if you want to check scanning page directly)"%(ingestoldcand_start))
failed = pl.ingest_avro(toingest)
logger.info("Tried to ingest %s, failed for %s"%(toingest,failed))
ingestoldcand_end = Time.now()
print("Stoppedingesting at %s (useful if you want to check scanning page directly)"%(ingestoldcand_end))

INFO:jnordin:Trying to ingest ZTF18aampiwv through avroid 635142595715015003
INFO:jnordin:... already a candidate with id 636222741415015023
INFO:jnordin:Trying to ingest ZTF18abmmszc through avroid 636309501015010006
INFO:jnordin:... already a candidate with id 636309501015010006
INFO:jnordin:Trying to ingest ZTF18abbmenz through avroid 634180094915015018
INFO:jnordin:... already a candidate with id 636199644915015015
INFO:jnordin:Trying to ingest ZTF18abvxqtp through avroid 636302985215015010
INFO:jnordin:... already a candidate with id 636303905215015008
INFO:jnordin:Trying to ingest ZTF18abwzedh through avroid 636377563715015010
INFO:jnordin:... already a candidate with id 636377563715015010
INFO:marshaltools.ProgramList:Trying to ingest 0 candidate(s) to to marhsal program AMPEL Test using ingest ID 4


Starting ingesting at 2018-09-29 18:57:45.719926 (useful if you want to check scanning page directly)


UnboundLocalError: local variable 'failed' referenced before assignment

In [16]:
# We do this again because it is soo funny
pl = marshaltools.ProgramList('AMPEL Test',load_candidates=True)

INFO:marshaltools.ProgramList:Initialized ProgramList for program AMPEL Test (ID 4)
INFO:marshaltools.ProgramList:Loaded 208 saved sources for program AMPEL Test.
INFO:marshaltools.ProgramList:Getting scanning page transient for program AMPEL Test between 2018-03-01 00:00:00 and 2018-09-29 18:58:33 using dt: 120.00 h
INFO:marshaltools.ProgramList:Downloading candidates. Iteration number 0: 43 jobs to do
INFO:marshaltools.ProgramList:jobs are done: total 43, failed: 0
INFO:marshaltools.ProgramList:Fetched 3555 candidates in 3.63e+01 sec


In [17]:
# And we do this again as well. If this worked we should now have all candidates saved
togrowth = []
for snnbr, choice in decisions.items():
    if not choice=='Submit' : continue

    canddate = pl.find_source(sncandidates[snnbr],include_candidates=True)
    if not canddate is None:
        continue
    avroid =  np.max( df_phot['_id'][ df_phot["ztf_name"]==sncandidates[snnbr] ]  )
    msg = "Failed to ingest alert %s for SN %s"%(avroid,sncandidates[snnbr])
    logger.info(msg)
    togrowth.append(msg)
    


In [18]:
# Write an email to Mansi/Growth team with the following content
print(togrowth)
# And when we hear back from them this SN should be saved. I hope they do it for us

[]


In [27]:
# NOTE: This regularly failes to be recieved at the Marshal ends. As long as one or more
# SNe are not properly saved you will have to rerun these cells with some breaks inbetween.
# At the other hand we can now saved based on the name, so it does not look like we have to worry about missing ids
missing_candidsne = []
# Now it is time to save remaining candidates
savecand_start = Time.now()
print("Starting saving at %s (not really that useful)"%(savecand_start))
for snnbr, choice in decisions.items():
    if not choice=='Submit' : continue

    try:
        candid = soupids[sncandidates[snnbr]]
        logger.info("Try to save %s through candid %s"%(sncandidates[snnbr],candid))
        pl.save_source_name(sncandidates[snnbr],42)
#        pl.save_source(candid,42)
    except KeyError:
        logger.info("Lacking candid for %s"%(sncandidates[snnbr]))
        missing_candidsne.append(snnbr)


        
savecand_end = Time.now()
print("Stopped saving at %s"%(savecand_end))

INFO:jnordin:Try to save ZTF18aampiwv through candid 1545342
INFO:marshaltools.ProgramList:Saving source ZTF18aampiwv into program 42


Starting saving at 2018-09-29 19:05:12.266935 (not really that useful)


INFO:jnordin:Try to save ZTF18abmmszc through candid 1541678
INFO:marshaltools.ProgramList:Saving source ZTF18abmmszc into program 42
INFO:jnordin:Try to save ZTF18abbmenz through candid 1545046
INFO:marshaltools.ProgramList:Saving source ZTF18abbmenz into program 42
INFO:jnordin:Try to save ZTF18abvxqtp through candid 1558718
INFO:marshaltools.ProgramList:Saving source ZTF18abvxqtp into program 42
INFO:jnordin:Try to save ZTF18abwzedh through candid 1564237
INFO:marshaltools.ProgramList:Saving source ZTF18abwzedh into program 42


Stopped saving at 2018-09-29 19:05:31.209390


In [28]:
# Check whether we have missing candidate ids
print("Missing candidate id for these sne: %s"%(missing_candidsne))

Missing candidate id for these sne: []


In [30]:
# Reload saved sources
pl = marshaltools.ProgramList('AMPEL Test',load_candidates=False)

INFO:marshaltools.ProgramList:Initialized ProgramList for program AMPEL Test (ID 4)
INFO:marshaltools.ProgramList:Loaded 213 saved sources for program AMPEL Test.


In [31]:
# Loop through them again to check they are all saved
for snnbr, choice in decisions.items():
    if not choice=='Submit' : continue

    canddate = pl.find_source(sncandidates[snnbr],include_candidates=False)
    if canddate is None:
        logger.info('SN %s was not saved. Oj oj.'%(sncandidates[snnbr]))
        continue

    logger.info('SN %s was saved'%(sncandidates[snnbr]))


INFO:jnordin:SN ZTF18aampiwv was saved
INFO:jnordin:SN ZTF18abmmszc was saved
INFO:jnordin:SN ZTF18abbmenz was saved
INFO:jnordin:SN ZTF18abvxqtp was saved
INFO:jnordin:SN ZTF18abwzedh was saved


In [ ]:
## Hopefully done. If all of the candidates were saved (no ojojs) you are done with scanning!
# If candidate IDs are still missing you have to try to scrape the webpages for them as below. Have fun!

In [23]:
# Trying to get candid through the seargant
from ampel.pipeline.t3.sergeant import marshal_functions
tbase = Time.now()
allcandidates = []

In [24]:
for k in range(1):
    print(k)
    tend = Time(tbase.jd-k,format='jd')
    tstart = Time(tbase.jd-k-1,format='jd')
    ser = marshal_functions.Sergeant(program_name='AMPEL Test',start_date=tstart.iso,end_date=tend.iso, marshalusr='jnordin',marshalpwd='oskarthebaseplayer')
    ser.cutprogramidx = 42
    cands = ser.list_scan_sources()
    print("Found %s cands"%(len(cands)))
    allcandidates.extend(cands)

0
Found 191 cands


In [25]:
for cand in allcandidates:
    if cand["name"] in soupids.keys():
        continue
    print("new candidate %s"%(cand["name"]))
    soupids[cand["name"]] = cand["candid"]

new candidate ZTF18abuqmxl
new candidate ZTF18abwkvem
new candidate ZTF18abvsvjs
new candidate ZTF18aboemtu
new candidate ZTF18abvttfl
new candidate ZTF18abyloto
new candidate ZTF18abzpbpi
new candidate ZTF18acafzqg
new candidate ZTF18abxjekh
new candidate ZTF18acagagq
new candidate ZTF18abuhyjv
new candidate ZTF18abxiaql
new candidate ZTF18abycxao
new candidate ZTF18acaekjn
new candidate ZTF18abtmbaz
new candidate ZTF18abwzedh
new candidate ZTF18abwkrbl
new candidate ZTF18abyzlhn
new candidate ZTF18absclsr
new candidate ZTF18abzsdju
new candidate ZTF18abzsllv
new candidate ZTF18aaadrhw
new candidate ZTF18abxroxl
new candidate ZTF18abybuhw
new candidate ZTF18abryqnn
new candidate ZTF18abodiov
new candidate ZTF18acafdth
new candidate ZTF18abznvol
new candidate ZTF18abyanoj
new candidate ZTF18abvyvlr
new candidate ZTF18abwfhqb
new candidate ZTF18abxdvvr
new candidate ZTF18abomklq
new candidate ZTF18acaednl
new candidate ZTF18abkhqww
new candidate ZTF18abcvcgs
new candidate ZTF18abkmfaj
n

In [26]:
# Save the added dict of candidate id
with open('sncandid_soup.json', 'w') as fp:
    json.dump(soupids, fp)

In [11]:
foo = """ZTF18ablrndb 
ZTF18aawlhkh
ZTF18abporae
ZTF17aabihdn
ZTF18aawlhkh
ZTF18absgvqb
ZTF18ablprcf
ZTF18abufaej
ZTF18absrcps
ZTF18abvfecb
ZTF18abwbtco
ZTF18abxdkni
ZTF18abvrzqj"""

In [12]:
foo2 = foo.split("\n")

In [13]:
print(foo2)

['ZTF18ablrndb ', 'ZTF18aawlhkh', 'ZTF18abporae', 'ZTF17aabihdn', 'ZTF18aawlhkh', 'ZTF18absgvqb', 'ZTF18ablprcf', 'ZTF18abufaej', 'ZTF18absrcps', 'ZTF18abvfecb', 'ZTF18abwbtco', 'ZTF18abxdkni', 'ZTF18abvrzqj']


In [17]:
for k,name in enumerate(sncandidates):
    print("%s %s"%(k,name))
    if name in foo2:
        decisions[k]="Submit"
    else:
        decisions[k]='foo'


0 ZTF18abslxpd
1 ZTF18ablrndb
2 ZTF18abtqrwa
3 ZTF18abgrlpv
4 ZTF17aabihdn
5 ZTF18aawlhkh
6 ZTF18abporae
7 ZTF18abulqzb
8 ZTF18abjhatv
9 ZTF18abnvirx
10 ZTF18abtlqub
11 ZTF18aaupjtd
12 ZTF18abtjrbt
13 ZTF18abgoodz
14 ZTF18absgvqb
15 ZTF18aajswer
16 ZTF18abmwfxd
17 ZTF18ablqjws
18 ZTF18abmarax
19 ZTF18aamnrdd
20 ZTF18abmxatl
21 ZTF18aayebnz
22 ZTF18aagteoy
23 ZTF18aampjcm
24 ZTF18aampiwv
25 ZTF18absljwl
26 ZTF18aaqzomm
27 ZTF18abslbyb
28 ZTF18aamntui
29 ZTF18abbmenz
30 ZTF18abmjokl
31 ZTF18abtghbi
32 ZTF18abnvotk
33 ZTF18aaxudip
34 ZTF18abujobx
35 ZTF18ablprcf
36 ZTF18absnsqx
37 ZTF18ababqeh
38 ZTF18aajupnt
39 ZTF18abflqxq
40 ZTF18abkhqww
41 ZTF18abufaej
42 ZTF18absrcps
43 ZTF18aawhlgv
44 ZTF18abygdtf
45 ZTF18abulemm
46 ZTF18abvxdou
47 ZTF18abxcbny
48 ZTF18abxcues
49 ZTF18abxxvza
50 ZTF18aavoyoq
51 ZTF18abxyudf
52 ZTF18abfsglv
53 ZTF18abecggb
54 ZTF18abvfecb
55 ZTF18abvmqnk
56 ZTF18abwbtco
57 ZTF18abzacuc
58 ZTF18abwqbxc
59 ZTF18abxdkni
60 ZTF18abxdtyz
61 ZTF18abvrzqj
62 ZTF18abymhbx
63

In [23]:
for snname in foo2:

    try:
        candid = soupids[snname]
        logger.info("Try to save %s through candid %s"%(snname,candid))
        pl.save_source(candid,42)
    except KeyError:
        logger.info("Lacking candid for %s"%(snname))


INFO:jnordin:Lacking candid for ZTF18ablrndb 
INFO:jnordin:Try to save ZTF18aawlhkh through candid 1545871
INFO:marshaltools.ProgramList:Saving source 1545871 into program 42
ERROR:marshaltools.ProgramList:No json returned: status 200
INFO:jnordin:Try to save ZTF18abporae through candid 1554627
INFO:marshaltools.ProgramList:Saving source 1554627 into program 42
ERROR:marshaltools.ProgramList:No json returned: status 200
INFO:jnordin:Try to save ZTF17aabihdn through candid 1546760
INFO:marshaltools.ProgramList:Saving source 1546760 into program 42
ERROR:marshaltools.ProgramList:No json returned: status 200
INFO:jnordin:Try to save ZTF18aawlhkh through candid 1545871
INFO:marshaltools.ProgramList:Saving source 1545871 into program 42
ERROR:marshaltools.ProgramList:No json returned: status 200
INFO:jnordin:Try to save ZTF18absgvqb through candid 1559399
INFO:marshaltools.ProgramList:Saving source 1559399 into program 42
ERROR:marshaltools.ProgramList:No json returned: status 200
INFO:jnor

In [26]:
for snname in foo2:
    canddate = pl.find_source(snname,include_candidates=False)
    if canddate is None:
        logger.info('SN %s was not saved. Oj oj.'%(snname))
        continue

    logger.info('SN %s was saved'%(snname))

INFO:jnordin:SN ZTF18ablrndb  was not saved. Oj oj.
INFO:jnordin:SN ZTF18aawlhkh was not saved. Oj oj.
INFO:jnordin:SN ZTF18abporae was saved
INFO:jnordin:SN ZTF17aabihdn was not saved. Oj oj.
INFO:jnordin:SN ZTF18aawlhkh was not saved. Oj oj.
INFO:jnordin:SN ZTF18absgvqb was saved
INFO:jnordin:SN ZTF18ablprcf was saved
INFO:jnordin:SN ZTF18abufaej was saved
INFO:jnordin:SN ZTF18absrcps was not saved. Oj oj.
INFO:jnordin:SN ZTF18abvfecb was saved
INFO:jnordin:SN ZTF18abwbtco was saved
INFO:jnordin:SN ZTF18abxdkni was saved
INFO:jnordin:SN ZTF18abvrzqj was saved


In [28]:
for snname in foo2:
    try:
        print('%s %s'%(snname,soupids[snname]))
    except KeyError:
        print("noooo.. %s"%(snname))

noooo.. ZTF18ablrndb 
ZTF18aawlhkh 1545871
ZTF18abporae 1554627
ZTF17aabihdn 1546760
ZTF18aawlhkh 1545871
ZTF18absgvqb 1559399
ZTF18ablprcf 1555641
ZTF18abufaej 1559090
ZTF18absrcps 1535950
ZTF18abvfecb 1556357
ZTF18abwbtco 1556215
ZTF18abxdkni 1554861
ZTF18abvrzqj 1558952


In [31]:
print(decisions)

{0: 'Garbage', 1: 'Wait', 2: 'Garbage', 3: 'Garbage', 4: 'Garbage', 5: 'Submit', 6: 'Submit', 7: 'Wait', 8: 'Submit', 9: 'Garbage', 10: 'Garbage', 11: 'Wait', 12: 'Wait', 13: 'Garbage', 14: 'Garbage', 15: 'Garbage', 16: 'Garbage', 17: 'Submit', 18: 'Garbage', 19: 'Garbage', 20: 'Garbage', 21: 'Garbage', 22: 'Garbage', 23: 'Wait', 24: 'Wait', 25: 'Garbage', 26: 'Garbage', 27: 'Garbage', 28: 'Garbage', 29: 'Submit', 30: 'Garbage', 31: 'Garbage', 32: 'Wait', 33: 'Garbage', 34: 'Garbage', 35: 'Garbage', 36: 'Garbage', 37: 'Garbage', 38: 'Garbage', 39: 'Garbage', 40: 'Wait', 41: 'Submit', 42: 'Submit', 43: 'Submit', 44: 'Garbage', 45: 'Submit', 46: 'Garbage', 47: 'Wait', 48: 'Garbage', 49: 'Garbage'}


In [9]:
decisions = {0: 'Garbage', 1: 'Wait', 2: 'Garbage', 3: 'Garbage', 4: 'Garbage', 5: 'Submit', 6: 'Submit', 7: 'Wait', 8: 'Submit', 9: 'Garbage', 10: 'Garbage', 11: 'Wait', 12: 'Wait', 13: 'Garbage', 14: 'Garbage', 15: 'Garbage', 16: 'Garbage', 17: 'Submit', 18: 'Garbage', 19: 'Garbage', 20: 'Garbage', 21: 'Garbage', 22: 'Garbage', 23: 'Wait', 24: 'Wait', 25: 'Garbage', 26: 'Garbage', 27: 'Garbage', 28: 'Garbage', 29: 'Submit', 30: 'Garbage', 31: 'Garbage', 32: 'Wait', 33: 'Garbage', 34: 'Garbage', 35: 'Garbage', 36: 'Garbage', 37: 'Garbage', 38: 'Garbage', 39: 'Garbage', 40: 'Wait', 41: 'Submit', 42: 'Submit', 43: 'Submit', 44: 'Garbage', 45: 'Submit', 46: 'Garbage', 47: 'Wait', 48: 'Garbage', 49: 'Garbage'}


In [67]:
bla = []
for snnbr, choice in decisions.items():
    if not choice=='Submit' : continue
        
    bla.append(sncandidates[snnbr])
    print('http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=%s'%(sncandidates[snnbr]))
print(bla)

http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18acvgjqv
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18acvwcky
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18acvwdkk
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18acvwmpw
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18acwwods
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18acwyaws
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18acxgqxq
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18acxhqur
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18acywbgz
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18acyxnyw
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18acyxock
http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=ZTF18acyyefr
http://skipper.caltech.edu:8

In [ ]:
ZTF18acvwdkk
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18acvwdkk&channel=HU_GP_CLEAN
ZTF18acwwods
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18acwwods&channel=HU_GP_CLEAN
ZTF18acwyaws
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18acwyaws&channel=HU_GP_CLEAN
ZTF18acxgqxq
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18acxgqxq&channel=HU_GP_CLEAN
ZTF18acxhqur
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18acxhqur&channel=HU_GP_CLEAN
ZTF18acywbgz
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18acywbgz&channel=HU_GP_CLEAN
ZTF18aczbkjn
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18aczbkjn&channel=HU_GP_CLEAN
ZTF18aczqtpt
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18aczqtpt&channel=HU_GP_CLEAN
ZTF18aczqzrj
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18aczqzrj&channel=HU_GP_CLEAN
ZTF18aczrafz
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18aczrafz&channel=HU_GP_CLEAN
ZTF18adacosi
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18adacosi&channel=HU_GP_CLEAN
ZTF18adaruzw
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18adaruzw&channel=HU_GP_CLEAN
ZTF18adayifh
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18adayifh&channel=HU_GP_CLEAN
ZTF18adaylyq
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18adaylyq&channel=HU_GP_CLEAN
ZTF18adazckv
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18adazckv&channel=HU_GP_CLEAN
ZTF18adazgdh
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18adazgdh&channel=HU_GP_CLEAN
ZTF18adbibui
https://www-zeuthen.desy.de/~jvsanten/ampel/FrontEnd/lc.html?target=ZTF18adbibui&channel=HU_GP_CLEAN